In [ ]:
import pandas as pd
#from os.path import dirname
import ast
from collections import defaultdict
#import seaborn as sns
import matplotlib.pyplot as plt
#from matplotlib.widgets import Slider
import pickle
#from PyMultiDictionary import MultiDictionary
#dictionary=PyDictionary()
#from itables import init_notebook_mode
#init_notebook_mode(all_interactive=True)


# Importing Data

In [ ]:
g = open('goodreads_preprocessed','rb')
goodreads = pickle.load(g)
g.close()

i = open('imdb_preprocessed','rb')
imdb = pickle.load(i)
i.close()

# Functions and Mappings

In [ ]:
mapping = {'movie': 'movie',
           'tvSeries':'tv',
           'tvMiniSeries':'tv',
           'video':'tv', 
           'short':'tv', 
           'videoGame':'tv',
           'tvMovie':'movie', 
           'tvEpisode':'tv', 
           'tvSpecial':'tv', 
           'tvShort':'tv'}

## Genres

### Goodreads

In [ ]:
goodreads['Genres'].head()

In [ ]:
goodreads['Genres'].head(10)[0]

In [ ]:
## Checking Genre frequency and sorting descendingly.

genres = defaultdict(int)
for lis in goodreads['Genres']:
    for x in ast.literal_eval(lis):
        genres[x] += 1
        
sorted_genres_g = dict(sorted(genres.items(), key=lambda x: x[1], reverse=True))
sorted_genres_g

In [ ]:
file = open('goodreads_genres_freq','ab')
pickle.dump(sorted_genres_g,file)
file.close()

# file = open('goodreads_genres_freq','rb')
# sorted_genres_g = pickle.load(file)
# file.close()


In [ ]:
## Functions aiding visualizations

def filter_func1(pair):
    k,v = pair
    if v>399:
        return (k,v)
    
def filter_func2(pair):
    k,v = pair
    if v<400 and v>100:
        return (k,v)
    
def filter_func3(pair):
    k,v = pair
    if v<101 and v>30:
        return (k,v)

In [ ]:
sorted_genres_filt1 = dict(filter(filter_func1,sorted_genres_g.items()))
genres_df1 = pd.DataFrame({'Genre': list(sorted_genres_filt1.keys()), 'Count': list(sorted_genres_filt1.values())})

sns.set(style="whitegrid")
plt.figure(figsize=(24, 6))
ax = sns.barplot(x="Genre", y="Count", data=genres_df1, palette="muted")
plt.title("Count of Genres with frequency greater than 400")
plt.xlabel("Genres")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.show()


In [ ]:
sorted_genres_filt2 = dict(filter(filter_func2,sorted_genres_g.items()))
genres_df2 = pd.DataFrame({'Genre': list(sorted_genres_filt2.keys()), 'Count': list(sorted_genres_filt2.values())})

sns.set(style="whitegrid")
plt.figure(figsize=(27, 6))
ax = sns.barplot(x="Genre", y="Count", data=genres_df2, palette="muted")
plt.title("Count of Genres with frequency between 100 and 400")
plt.xlabel("Genres")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.show()

In [ ]:
sorted_genres_filt3 = dict(filter(filter_func3,sorted_genres_g.items()))
genres_df3 = pd.DataFrame({'Genre': list(sorted_genres_filt3.keys()), 'Count': list(sorted_genres_filt3.values())})

sns.set(style="whitegrid")
plt.figure(figsize=(27, 6))
ax = sns.barplot(x="Genre", y="Count", data=genres_df3, palette="muted")
plt.title("Count of Genres with frequency less than 100")
plt.xlabel("Genres")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# These will be our columns for the Genre Matrix
genre_matrix_cols_g = list(sorted_genres_g.keys())

matrix = []
for i in range(len(goodreads)):
    lis_genres_per_row = ast.literal_eval(goodreads.iloc[i,3])

    matrix_row = []
    for colName in genre_matrix_cols_g:
        if colName in lis_genres_per_row:
            matrix_row.append(1)
        else:
            matrix_row.append(0)
    matrix.append(matrix_row)

genre_matrix_df_g = pd.DataFrame(matrix,columns=genre_matrix_cols_g)
genre_matrix_df_g.head()


In [ ]:
genre_matrix_freq_df_g = genre_matrix_df_g.apply(pd.Series.value_counts)
display(genre_matrix_freq_df_g)

In [ ]:
genre_matrix_freq_df_T_g = genre_matrix_freq_df_g.transpose()
genre_matrix_freq_df_T_g.rename(columns={0:'0',1:'1'},inplace=True)
genre_matrix_freq_df_T_g

In [ ]:

ax = genre_matrix_freq_df_g.transpose().head(10).plot(kind='bar', width=0.4, align='center', figsize=(20,10))
ax.set_xticklabels(genre_matrix_freq_df_g.transpose().head(10).index, rotation=45, ha='right')
# Customize the plot
ax.set_xlabel('Genres')
ax.set_ylabel('Frequency')
ax.set_title('Double Bar Graph of Genre Frequencies (0 and 1)')
ax.legend(title='Value', labels=['0', '1'])

# Show the plot
plt.show()


In [ ]:
import pandas as pd

# Create the DataFrame
data = matrix

columns = genre_matrix_df_g.columns

df = pd.DataFrame(data, columns=columns)

# Calculate the correlation matrix
correlation_matrix = df.corr()

#display(correlation_matrix)
correlation_matrix.style.background_gradient(cmap='coolwarm')

In [ ]:
plt.figure(figsize=(10, 5), dpi=200)
sns.heatmap(correlation_matrix)

In [ ]:
def per_to_int(x):
    return ast.literal_eval(x[:5])

In [ ]:
genre_matrix_freq_df_T_g['1'].apply(lambda x: '{:,.2%}'.format(x/10000))

In [ ]:
genre_matrix_freq_df_T_g['1'].loc[lambda x: x/10000<0.01]

In [ ]:
sns.boxplot(data=genre_matrix_freq_df_T_g['1'])

### IMDB

In [ ]:
imdb['genres'].head()

In [ ]:
imdb['genres'].value_counts().head()

In [ ]:
imdb['genres'] = imdb['genres'].apply(lambda x: list(map(str,x.split(','))))

In [ ]:
imdb.head()

In [ ]:
genres = defaultdict(int)
for lis in imdb['genres']:
    for x in lis:
        genres[x] += 1
        
sorted_genres_i = dict(sorted(genres.items(), key=lambda x: x[1], reverse=True))
sorted_genres_i

In [ ]:
file = open('imdb_genres_freq','ab')
pickle.dump(sorted_genres_i,file)
file.close()

# file = open('goodreads_genres_freq','rb')
# sorted_genres_g = pickle.load(file)
# file.close()

In [ ]:
#sorted_genres_filt1 = dict(filter(filter_func1,sorted_genres.items()))
genres_df1 = pd.DataFrame({'Genre': list(sorted_genres_i.keys()), 'Count': list(sorted_genres_i.values())})

sns.set(style="whitegrid")
plt.figure(figsize=(24, 12))
ax = sns.barplot(x="Genre", y="Count", data=genres_df1, palette="muted")
plt.title("Count of Genres with frequency greater than 400")
plt.xlabel("Genres")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# These will be our columns for the Genre Matrix
genre_matrix_cols_i = list(sorted_genres_i.keys())

matrix_i = []
for i in range(len(imdb)):
    lis_genres_per_row = imdb.iloc[i,5]

    matrix_row = []
    for colName in genre_matrix_cols_i:
        if colName in lis_genres_per_row:
            matrix_row.append(1)
        else:
            matrix_row.append(0)
    matrix_i.append(matrix_row)

genre_matrix_df_i = pd.DataFrame(matrix_i,columns=genre_matrix_cols_i)
genre_matrix_df_i.head()


In [ ]:
genre_matrix_freq_df_i = genre_matrix_df_i.apply(pd.Series.value_counts)
genre_matrix_freq_df_i

In [ ]:
genre_matrix_freq_df_T_i = genre_matrix_freq_df_i.transpose()
genre_matrix_freq_df_T_i.rename(columns={0:'0',1:'1'},inplace=True)
genre_matrix_freq_df_T_i

In [ ]:
data = matrix_i
columns = genre_matrix_df_i.columns
df = pd.DataFrame(data, columns=columns)
correlation_matrix = df.corr()
correlation_matrix.style.background_gradient(cmap='coolwarm')

# next

## 1

In [ ]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# Sample genre lists
movies_genres = genre_matrix_cols_i
books_genres = genre_matrix_cols_g

# Combine both genre lists
combined_genres = movies_genres + books_genres

# Train a Word2Vec model on the combined genres
model = Word2Vec([combined_genres], vector_size=100, window=5, min_count=1, sg=0)

In [ ]:
# Find similar genres based on cosine similarity of embeddings
similar_genres = model.wv.most_similar("Fiction", topn=5)  # Replace "Action" with the genre of interest

print("Similar genres:")
for genre, score in similar_genres:
    print(genre, score)

## 2

In [ ]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Example embeddings for "True Story" and "Fiction" using a trained Word2Vec model
word_embeddings = {
    "True Story": np.array([0.5, 0.7, 0.2, ...]),  # Replace with actual embeddings
    "Fiction": np.array([-0.3, -0.5, 0.8, ...])  # Replace with actual embeddings
}

# Compute cosine similarity
similarity_score = cosine_similarity([word_embeddings["True Story"]], [word_embeddings["Fiction"]])[0][0]

print(f"Semantic similarity score between 'True Story' and 'Fiction': {similarity_score}")

## 3

In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('wordnet_ic')

In [ ]:
from sematch.semantic.similarity import WordNetSimilarity
wns = WordNetSimilarity()
wns.word_similarity('dog', 'cat', 'li')

## 4

In [ ]:
print(spacy.__version__)

In [ ]:
import spacy

# Load the pre-trained model
nlp = spacy.load("en_core_web_sm")

# Calculate semantic similarity
doc1 = nlp("Made up Story")
doc2 = nlp("Fiction")

similarity_score = doc1.similarity(doc2)

print(f"Semantic similarity score between 'True Story' and 'Fiction': {similarity_score}")

## 5

In [ ]:
genre_matrix_freq_df_T_g

In [ ]:
genre_matrix_freq_df_T_i.index

In [ ]:
genre_matrix_freq_df_T_g.index

### Searching genre function

In [ ]:
import re

In [ ]:
def search_genre(to_search,type='Goodreads'):
    if type=='Goodreads':
        lis = list(genre_matrix_freq_df_T_g.index)
    elif type=='IMDB':
        lis = list(genre_matrix_freq_df_T_i.index)
    
    matches = [word for word in lis if re.search(to_search.lower(), word.lower())]
    return matches

### Finding synonyms within goodreads genres

In [ ]:
search_genre('myth')

In [ ]:
print(dictionary.synonym('en', 'fiction'))

In [ ]:
for g_genre in genre_matrix_freq_df_T_g.index:
    lis = list(genre_matrix_freq_df_T_g.index)
    temp_lis = lis.copy()
    temp_lis.remove(g_genre)
    syn_lis = dictionary.synonym('en', g_genre.lower())
    genre_synonyms = [other for other in temp_lis if other.lower() in syn_lis]
    break

genre_synonyms

In [ ]:
whole_lis = {}
for g_genre in genre_matrix_freq_df_T_i.index:
    lis = list(genre_matrix_freq_df_T_i.index)
    glis = list(genre_matrix_freq_df_T_g.index)
    syn_lis = dictionary.synonym('en', g_genre.lower())
    whole_lis[g_genre] = [other for other in glis if other.lower() in syn_lis]

whole_lis

In [ ]:
dictionary = MultiDictionary()
print(dictionary.synonym('en', 'Fantasy'))

# rest

In [ ]:
import nltk
from nltk.corpus import wordnet
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('wordnet')

In [ ]:
def synonym_antonym_extractor(phrase):
     synonyms = []
     antonyms = []
     for syn in wordnet.synsets(phrase):
          for l in syn.lemmas():
               synonyms.append(l.name())
               if l.antonyms():
                    antonyms.append(l.antonyms()[0].name())

     print(set(synonyms))
     print(set(antonyms))

synonym_antonym_extractor(phrase="Drama")

* fiction / non - fiction should be atlow weightage


In [ ]:
goodreads.head()

In [ ]:
goodreads['Genres'].head(10)[0]

In [ ]:
imdb.isin(['To Kill a Mockingbird']).any()


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Define your sentences
sentence1 = "True Story"
sentence2 = "Fiction"

# Encode the sentences into embeddings
embeddings = model.encode([sentence1, sentence2], convert_to_tensor=True)

# Calculate cosine similarity
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])

# Print the similarity score
print(f"Semantic similarity score between '{sentence1}' and '{sentence2}': {cosine_scores.item()}")
